In [6]:
# !sudo yum install Xvfb -y -y
# !pip install scrapy
# !pip install fake_useragent
# !pip install scrapy-selenium
# !pip install selenium==4.9.1
# !pip install boto3==1.28.5
# !pip install crochet
# !pip install awslambdaric
# !pip install botocore==1.31.10


# import os
# cmd = '''
# sudo yum install atk cups-libs gtk3 libXcomposite alsa-lib libXcursor libXdamage libXext libXi libXrandr libXScrnSaver libXtst pango at-spi2-atk libXt xorg-x11-server-Xvfb xorg-x11-xauth dbus-glib dbus-glib-devel -y
# '''
# os.system(cmd)


# !curl  -Lo "/tmp/chromedriver.zip" "https://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_linux64.zip"
# !unzip /tmp/chromedriver.zip -d /opt/ -y
# !sudo ln -sf /opt/chromedriver /usr/local/bin/chromedriver

# !curl -Lo "/tmp/chrome-linux.zip" "https://www.googleapis.com/download/storage/v1/b/chromium-browser-snapshots/o/Linux_x64%2F1135561%2Fchrome-linux.zip?alt=media"
# !unzip /tmp/chrome-linux.zip -d /opt/ -y
# !sudo ln -sf /opt/chrome-linux/chrome /usr/local/bin/chrome



In [7]:
import datetime
import json
import os
import botocore
from fake_useragent import UserAgent
import boto3
from scrapy.crawler import CrawlerRunner
from scrapy.utils.project import get_project_settings
from crochet import setup
import threading
# from momoshop_link import MomoshopSpider

# Initialize Crochet
setup()

ua = UserAgent()
CHROME_BROWSER_PATH = '/usr/local/bin/chrome'
region = 'ap-northeast-1'
sqs = boto3.client('sqs', region_name=region, config=botocore.client.Config(max_pool_connections=500))
queue_tier4_links = 'https://sqs.ap-northeast-1.amazonaws.com/407620147666/aws_crawler_tier4_links.fifo'

In [11]:
import hashlib
import json
import os
import threading
from urllib.parse import urlparse

from scrapy_selenium import SeleniumRequest
import botocore
import scrapy
from fake_useragent import UserAgent
import boto3
from selenium.common.exceptions import NoAlertPresentException, UnexpectedAlertPresentException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.wait import WebDriverWait
from selenium import webdriver

ua = UserAgent()
region = 'ap-northeast-1'
CHROME_BROWSER_PATH = '/usr/local/bin/chrome'
sqs = boto3.client('sqs', region_name=region, config=botocore.client.Config(max_pool_connections=500))
s3 = boto3.client('s3', region_name=region, config=botocore.client.Config(max_pool_connections=500))
export_to_s3 = 'Off'
queue_tier4_content_links = 'https://sqs.ap-northeast-1.amazonaws.com/407620147666/aws_crawler_tier4_content_links.fifo'
s3_tier4_content_links = 'tier4-content-links'
s3_crawler_content_folder = 'aws-web-crawler-data'


def hash_function(original_string):
    # Create a hash object using SHA-256 algorithm
    hasher = hashlib.sha256()

    # Convert the input string to bytes (required by hashlib)
    input_bytes = original_string.encode('utf-8')

    # Update the hash object with the input bytes
    hasher.update(input_bytes)

    # Get the hexadecimal representation of the hash value
    return hasher.hexdigest()


class MomoshopSpider(scrapy.Spider):
    name = "momoshop"
    allowed_domains = ["www.momoshop.com.tw"]
    user_agent = ua.random
    runner = None
    category_links = []

    def __init__(self, runner=None, **kwargs):
        self.runner = runner
        super(MomoshopSpider, self).__init__(**kwargs)

    def start_requests(self):
        # Start the initial request to fetch category links
        print("Starting start_requests")
        if self.runner.target_url == "":
            target_url = json.loads(self.runner.tier4_category_object['Body'])['category_link']
        else:
            target_url = self.runner.target_url
        # target_url = "https://www.momoshop.com.tw/category/DgrpCategory.jsp?d_code=2001400023&p_orderType=1"
        headers = {"Host": urlparse(target_url).netloc,
                   'Accept-Encoding': 'gzip, deflate, br',
                   'Accept-Language': 'en-US,en;q=0.9,zh-TW;q=0.8,zh;q=0.7,zh-CN;q=0.6,ja;q=0.5',
                   'Sec-Fetch-Dest': 'document',
                   'Sec-Fetch-Mode': 'navigate',
                   'Sec-Fetch-Site': 'none',
                   'Upgrade-Insecure-Requests': '1',
                   "Referer": "https://www.momoshop.com.tw/main/Main.jsp",
                   "User-Agent": ua.random}
        print(f'target_url: {target_url}')
        yield SeleniumRequest(url=target_url,
                              headers=headers,
                              callback=self.parse,
                              wait_time=15,
                              errback=self.error_handle,
                              wait_until=ec.presence_of_element_located((By.XPATH, "//*")))

    def error_handle(self, failure):
        exception = failure.value
        print(exception)
        if isinstance(exception, UnexpectedAlertPresentException):
            print("Unexpected alert encountered. Handling...")
            # Perform any additional handling you need here
        else:
            self.runner.timeout = True
            print("Other error occurred:", exception)
            # Handle other types of errors here

    def parse(self, response, **kwargs):
        try:
            # Parse the main category page and extract links to individual product pages
            print(f"Starting to parse")

            # Get the number of total pages
            # driver = response.request.meta['driver']
            
            #-------------- add to avoid bugs from jupyter-lab --------------------
            options = webdriver.ChromeOptions()
            options.binary_location = "/opt/chrome-linux/chrome"
            options.add_argument("--headless")
            options.add_argument("--no-sandbox")
            options.add_argument("--single-process")
            options.add_argument("--disable-dev-shm-usage")
            options.add_argument("blink-settings=imagesEnabled=false")
            options.add_argument("--remote-debugging-port=9222")
            options.add_argument("--no-zygote")
            options.add_argument("--disable-gpu")
            options.add_argument("--disable-dev-tools")
    
            driver = webdriver.Chrome(executable_path = '/usr/local/bin/chromedriver', chrome_options=options)
            driver.get(response.url)
            
            wait = WebDriverWait(driver, 20)  # Adjust the timeout as needed
            wait.until(ec.presence_of_element_located((By.XPATH, "//*") ))
            updated_content = driver.page_source
            response = scrapy.http.HtmlResponse(url=driver.current_url, body=updated_content,
                                                        encoding='utf-8')
            #----------------------------------------------------------------
            
            
            # Get the updated page source and extract product links
            tier4_content_links = response.css(".eachGood ::attr(href)").getall()

            threads = []
            if len(tier4_content_links) > 0:
                tier = self.set_all_tier(response)
                # self.construct_s3_tier_folder(tier)
                self.start_to_push(tier4_content_links, tier, threads)
                
                page_element = response.css('.pageArea dl dt span ::text').getall()
                
                
                # Some pages have no page count element "頁數1/1"
                if len(page_element) > 0:
                    total_pages = int(response.css('.pageArea dl dt span ::text').getall()[-1].replace("/", ""))
                    
                    # To get the page index attribute
                    page_attrib = ""


                    for key, value in response.css('.pageArea a')[1].attrib.items():    #to skip bug 
                        try:
                            int(value)
                            page_attrib = key
                        except ValueError:
                            pass

                    if total_pages > 1:
                        for i in range(2, total_pages + 1):
                            # For each page, extract product links
                            # If the number of page is greater than 1 then
                            # For subsequent pages, click on the page number to load new content and extract links
                            js_code = f"document.querySelector('.pageArea a[{page_attrib}=\"{i}\"]').click();"  # Click the page number
                            driver.execute_script(js_code)
                            print(f'page: {i}')

                            # Wait for the new content to load after the click
                            wait = WebDriverWait(driver, 20)  # Adjust the timeout as needed
                            updated_content_locator = (By.XPATH, "//*")  # Wait for product content(class name 'eachGood') loaded
                            wait.until(ec.presence_of_element_located(updated_content_locator))

                            # Get the updated page source and extract product links
                            updated_content = driver.page_source
                            updated_response = scrapy.http.HtmlResponse(url=driver.current_url, body=updated_content,
                                                                        encoding='utf-8')
                            tier4_content_links = updated_response.css(".eachGood ::attr(href)").getall()  # Get all product url

                            # Start the threading to push the product links to SQS and/or S3
                            self.start_to_push(tier4_content_links, tier, threads)

            # Wait for all threads to finish before proceeding
            for thread in threads:
                thread.join()
        except Exception as e:
            print(str(e))

    @staticmethod
    def set_all_tier(response):
        # The original class array => ['Home', '家電', '飲水設備', '\n            ', '本月主打', '元山★開館慶下殺']
        tier_array = "|".join(response.css("#bt_2_layout_NAV ul li ::text").getall()).replace(" ", "").replace("\n", "").replace("||", "|").split("|")
        tier_array.pop(0)
        print(tier_array)
        return tier_array

    @staticmethod
    def construct_s3_tier_folder(tier: dict):
        # Construct the base folder path
        base_path = "content"

        # Create the base folder if it doesn't exist
        # if base_path:
            # s3.put_object(Bucket=s3_crawler_content_folder, Key=f"{base_path}/")

        # Construct the folder structure based on the tiers
        for key, value in tier.items():
            folder_name = value.strip('/')
            folder_path = f"{base_path}/{folder_name}"

            # Create the folder if it doesn't exist
            s3.put_object(Bucket=s3_crawler_content_folder, Key=f"{folder_path}/")

            # Set the current folder path as the base path for the next iteration
            base_path = folder_path

    def start_to_push(self, tier4_content_links, tier, threads):
        # For each product link, create a message and push it to SQS and/or S3
        for tier4_content_link in tier4_content_links:
            tier4_content_link = f'https://www.momoshop.com.tw/{tier4_content_link}'.replace("//", "/")
            hash_id = hash_function(tier4_content_link)
            message = {'Id': hash_id, 'MessageGroupId': hash_id,
                       'MessageBody': json.dumps({'tier4_content_link': tier4_content_link, 'tier': tier}, ensure_ascii=False)}
            print(message)
#             thread_sqs = threading.Thread(target=self.send_to_sqs, args=(message,))
#             threads.append(thread_sqs)
#             thread_sqs.start()

#             # self.send_to_sqs(message)

#             if export_to_s3 is not None and export_to_s3 == "On":
#                 message_body = {'hash_id': hash_id, 'page_url': tier4_content_link}
#                 self.send_to_s3(message_body, hash_id)
#                 thread_s3 = threading.Thread(target=self.send_to_sqs, args=(message,))
#                 threads.append(thread_s3)
#                 thread_s3.start()

#     @staticmethod
#     def send_to_sqs(message):
#         # Send the message to SQS
#         try:
#             # print(message['MessageGroupId'])
#             response = sqs.send_message(QueueUrl=queue_tier4_content_links,
#                                         MessageGroupId=message['MessageGroupId'],
#                                         MessageBody=message['MessageBody'],
#                                         MessageDeduplicationId=message['MessageGroupId'])
#             if response["ResponseMetadata"]["HTTPStatusCode"] != 200:
#                 print('Fail')
#         except Exception as e:
#             print(str(e))

#     @staticmethod
#     def send_to_s3(m, hash_id):
#         # Send the message to S3
#         try:
#             json_file_name = f'{hash_id}.json'
#             response = s3.put_object(Bucket=s3_tier4_content_links, Key=json_file_name,
#                                      Body=json.dumps(m, ensure_ascii=False))
#             # check if it's successful
#             if response["ResponseMetadata"]["HTTPStatusCode"] != 200:
#                 print('Fail')
#         except Exception as e:
#             print(str(e))

    def spider_closed(self):
        # Handle the spider closing event
        self.runner.timeout = False


In [12]:
class LambdaRunner:
    target_url = ""
    receipt_handle = ""
    tier4_category_object = None
    timeout = False

    def __init__(self, url):
        self.finished = threading.Event()
        self.results = []
        if url != "":
            self.target_url = url

    def run_spider(self):
        # Create a CrawlerRunner with project settings
        settings = get_project_settings()
        runner = CrawlerRunner(settings)
        if self.target_url == "":
            self.get_tier4_url_from_sqs()

        # Callback function to handle the spider results
        def handle_results(result):
            self.results.append(result)

            # Check if the spider has finished running
            if len(self.results) == 1:
                self.finished.set()

        # Start the first spider run
        deferred = runner.crawl(MomoshopSpider, runner=self)
        deferred.addCallback(handle_results)

        # Start the reactor
        runner.join()

    def wait_for_completion(self):
        self.finished.wait()

    def get_results(self):
        return self.results

    def get_tier4_url_from_sqs(self):
        response = sqs.receive_message(
            QueueUrl=queue_tier4_links,
            MaxNumberOfMessages=1,  # Retrieve 10 messages
            WaitTimeSeconds=0  # Maximum time to wait for messages (long polling)
        )
        messages = response.get('Messages', [])
        if messages is not None:
            self.tier4_category_object = messages[0]
        else:
            msg = "All consumed."
            print(msg)
            return msg

        # Delete messages from SQS
        for message in messages:
            sqs.delete_message(
                QueueUrl=queue_tier4_links,
                ReceiptHandle=message['ReceiptHandle']
            )


def handler(event, context):
    try:
        # Check if the function was triggered by an HTTP request or Lambda event
        print(f"Starting to crawl: {datetime.datetime.now()}")
        times = 0
        if "statusCode" not in event:
            # If the function was not triggered by retry
            runner = LambdaRunner("")
            runner.run_spider()
            runner.wait_for_completion()
            print(f"End date and time: {datetime.datetime.now()}")
        else:
            times = int(event["times"])
            if times < 4:
                runner = LambdaRunner(event["category_link"])
                runner.input_url = event["category_link"]
                runner.run_spider()
                runner.wait_for_completion()
                print(f"End date and time: {datetime.datetime.now()}")
            else:
                print(f'Retry too many times, 429: {event["category_link"]}')
                # If the retry count is 4 or more, return an HTTP 429 response indicating Too Many Requests
                return {
                    'statusCode': 429,
                    'body': "Retry too many times",
                    'times': times,
                    "category_link": event["category_link"]
                }

        times = times + 1
        if not runner.timeout:
            print('success')
            # If the LambdaRunner completed successfully, return an HTTP 200 response with the completion details
            return {
                'statusCode': 200,
                'body': 'Completed!',
                'times': times,
                # "category_link": event["category_link"],
                # "category_info": runner.category_info
            }
        else:
            print('timeout')
            # If the LambdaRunner timed out, return an HTTP 408 response with the category objects
            return {
                'statusCode': 408,
                'times': times,
                "category_link": json.loads(runner.tier4_category_object['Body'])['category_link']
            }
    except Exception as e:
        print(f'fail:{e}')
        return {
            'statusCode': 500,
            'body': json.dumps({'error': str(e)})
        }



In [13]:
if __name__ == '__main__':
    handler({'statusCode': 200,
             'times':1, 
             'category_link': 'https://www.momoshop.com.tw/category/DgrpCategory.jsp?d_code=2919101050&p_orderType=6&showType=chessboardType'}, 
            "")

Starting to crawl: 2023-08-25 02:13:04.093537
Starting start_requests
target_url: https://www.momoshop.com.tw/category/DgrpCategory.jsp?d_code=2919101050&p_orderType=6&showType=chessboardType
Starting to parse
['家電', '冰箱', '國際牌', '玻璃系列']
{'Id': '9589a96f41593c8c49cb35b0937be04c8593062326207162f84aebcd6611a02d', 'MessageGroupId': '9589a96f41593c8c49cb35b0937be04c8593062326207162f84aebcd6611a02d', 'MessageBody': '{"tier4_content_link": "https:/www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=11440666&str_category_code=2919101050&sourcePageType=4", "tier": ["家電", "冰箱", "國際牌", "玻璃系列"]}'}
{'Id': 'ecc88e5b36a586fdc2d1521e4fe307ba3e10e2d18433e63d2adaac4398668604', 'MessageGroupId': 'ecc88e5b36a586fdc2d1521e4fe307ba3e10e2d18433e63d2adaac4398668604', 'MessageBody': '{"tier4_content_link": "https:/www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=11440667&str_category_code=2919101050&sourcePageType=4", "tier": ["家電", "冰箱", "國際牌", "玻璃系列"]}'}
{'Id': 'ae100027000ec7cd01ea67402f108541a086943a674b3e7d